In [1]:
url = "https://plato.stanford.edu/contents.html"

In [2]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import requests
    
#open index page, store contents and close
uClient = urlopen(url)
page = uClient.read()
uClient.close()

#parse index page html
page_soup = soup(page, "html.parser")
    
#collect all text content in index page body
body = page_soup.body

#retrieve all entries, stored as keys in a dictionary
entries={}
for _ in body.find_all("a"):
    if str(_.get("href"))[:7]=="entries":
        entries[_.get("href")[8:-1]]=None

#iterate through each entry's page to retrieve content
for _ in entries:
    
    try:
    
        #adjust url for entry
        entry_url = url[:-13]+"entries/"+_+"/"

        #open, store, close entry page
        uClient = urlopen(entry_url)
        entry_page=uClient.read()
        uClient.close()

        #parse entry html
        entry_soup = soup(entry_page,"html.parser")
        entry_body = entry_soup.body

        #store content, add it as item to entry dict
        content=[]
        for text in body.find_all("p"):
            content.append(text)
            
        entries[_]=content
        
    except:
        pass

print (entries)

TypeError: unhashable type: 'slice'